In [23]:
from main import load_and_preprocess_data

df, encoders = load_and_preprocess_data()

Loading and preprocessing data...
Class distribution:
is_fraud
0    1289169
1       7506
Name: count, dtype: int64
Fraud percentage: 0.58%


In [25]:
from model import load_lstm_model
from predict import predict_and_analyze

# sample_cc_num = df['cc_num'].iloc[0]
sample_cc_num = 2703186189652095
user_transactions = df[df['cc_num'] == sample_cc_num]

last_transaction = user_transactions.iloc[-1].to_dict()

lstm_model, scaler, processor = load_lstm_model()

sequence_length = 10

results = predict_and_analyze(
    lstm_model,
    user_transactions.iloc[:-1],  # Use all but last transaction
    sequence_length,
    processor,
    actual_transaction=last_transaction
)

last_transaction 

Loading model from models/lstm_transaction_model.h5...


ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 10, 12), found shape=(1, 100, 12)